# Assignment done by Mukhtadir Syed

## Part 1 (regular expressions)

We need to extract dates from a txt file, the date are in all different formats

### Data loading

In [1]:
with open("./medical_dataset.txt") as f:
    lines = f.readlines()

lines

['03/25/93 Total time of visit (in minutes):\n',
 '6/18/85 Primary Care Doctor:\n',
 'sshe plans to move as of 7/8/71 In-Home Services: None\n',
 '7 on 9/27/75 Audit C Score Current:\n',
 '2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n',
 '.Per 7/06/79 Movement D/O note:\n',
 "4, 5/18/78 Patient's thoughts about current substance abuse:\n",
 '10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n',
 '3/7/86 SOS-10 Total Score:\n',
 '(4/10/71)Score-1Audit C Score Current:\n',
 '(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent\n',
 '4/09/75 SOS-10 Total Score:\n',
 '8/01/98 Communication with referring physician?: Done\n',
 '1/26/72 Communication with referring physician?: Not Done\n',
 '5/24/1990 CPT Code: 90792: With medical services\n',
 '1/25/2011 CPT Code: 90792: With medical services\n',
 '4/12/82 Total time of visit (in minutes):\n',
 '1; 10/13/1976 Audit C Score, Highest/Date:\n',
 '4, 4/24/98 Relevan

### At first we will import the required libraries and read the file as a pandas datafram

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(lines, columns=["text"])
df.head()

,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Services: None\n
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n


In [3]:
df.shape 
# We have 445 rows of text which has the dates in it

(445, 1)

## Task

1- Evaluate the file and check the dates format

2- Compactly write the regular expression to extract dates from every line

3- End result needs to be a data frame with 4 columns- original text, month, day , year


### Tips

* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* There could be potential typos as this is a raw, real-life derived dataset.

### Regex extracting

With the below regex expression we will capture the following patterns,
 - Month (1 or 2 digits) Day (1 or 2 digits) Year (2 to 4 digits)
 - Month, Day and Year can be separated by "/" or "-" or "space"

In [4]:
df_aux = df["text"].str\
         .extract(r'(?P<month>\d{1,2})[/\-\s](?P<day>\d{1,2})[/\-\s](?P<year>\d{2,4})').dropna(how = 'any')

Below will extract dates in the following pattern:
 - Day (1 or 2 digits) "space" Month (atleast one word character) "space" year with 4 digits

In [5]:
df_aux=pd.concat([df_aux,
         df[~df.index.isin(df_aux.index.tolist())]["text"]\
         .str.extract(r'(?P<day>\d{1,2})\s(?P<month>\w+)\s(?P<year>\d{4})').dropna(how = 'any')])

df_aux.sort_index(inplace = True)

In [6]:
df_aux.shape

(195, 3)

 - Now extracting months in word and year (4 digits) separated by either space or ","

In [7]:
df_aux = pd.concat([df_aux,
         df[~df.index.isin(df_aux.index.tolist())]["text"]\
         .str.extract(r'(?P<month>\w{3,10})[\,\s]*(?P<year>\d{4})').dropna(how = 'any')])

df_aux.sort_index(inplace = True)

In [8]:
df_aux.shape

(310, 3)

- Now extracting dates in Month as 1 or 2 digits followed by "/" and year with 4 digits

In [9]:
df_aux = pd.concat([df_aux,
         df[~df.index.isin(df_aux.index.tolist())]["text"]\
         .str.extract(r'(?P<month>\d{1,2})/(?P<year>\d{4})').dropna(how = 'any')])

df_aux.sort_index(inplace = True)

In [10]:
df_aux.shape

(411, 3)

Formats:
- MM dd yyyy (example: Jan 30 2022)
- MM dd, yyyy (example: Jan 30, 2022)
- MM. dd, yyyy (example: Jan. 30, 2022)

In [11]:
df_aux=pd.concat([df_aux,
         df[~df.index.isin(df_aux.index.tolist())]["text"]\
         .str.extract(r'(?P<month>\w+)[\.\s\,]*(?P<day>\d{1,2})[\.\s\,]*(?P<year>\d{4})').dropna(how = 'any')])
df_aux.sort_index(inplace = True)

Let us see now if there is anything left

In [12]:
df[~df.index.isin(df_aux.index.tolist())]

,text


The final dataframe with all the dates extracted is:

In [13]:
df_aux.shape

(445, 3)

In [14]:
df_aux.isna().sum()

month      0
day      216
year       0
dtype: int64

### Data Cleansing

As seen above there are some NAN values

In [15]:
df_aux.fillna(1, inplace = True)

In [16]:
df_aux.isna().sum()

month    0
day      0
year     0
dtype: int64

- Convert all the years so they have always four digits.

In [17]:
df_aux = df_aux.astype({"year":'int'})
df_aux["year"] = np.where(df_aux['year'] < 100, df_aux['year'] + 1900 , df_aux['year'])

## Further Clean up
- We are now going to clean up the data as expected and correct some mistakes in the data as shown below

In [18]:
#Row 283:

df_aux.loc[(df_aux["month"]=="Decemeber") & (df_aux["day"]==1) & (df_aux["year"]==1978)] = [["December", "1", 1978]]

#Row 298:

df_aux.loc[(df_aux["month"]=="Janaury") & (df_aux["day"]==1) & (df_aux["year"]==1993)] = [["January", "1", 1993]]

#Row 313:

df_aux.loc[(df_aux["month"]=="2June") & (df_aux["day"]==1) & (df_aux["year"]==1999)] = [["June", "2", 1999]]

#Row 392:

df_aux.loc[(df_aux["month"]=="67") & (df_aux["day"]=="5") & (df_aux["year"]==2000)] = [["5","1",2000]]

- Further checking the data shows that the following rows have issues 283, 256, 245, 235, 333, 285, 281.

In [19]:
list1 = [283, 256, 245, 235, 333, 285, 281]
[df_aux["month"].iloc[x] for x in list1]

['AFeb', 'IAug', 'lNovember', 'pOct', 'sNovember', 'sSep', 'yAug']

 - fixing the above mistakes

In [20]:
df_aux["month"] = df_aux["month"].str.replace("AFeb","Feb")
df_aux["month"] = df_aux["month"].str.replace("IAug","Aug")
df_aux["month"] = df_aux["month"].str.replace("lNovember","November")
df_aux["month"] = df_aux["month"].str.replace("pOct","Oct")
df_aux["month"] = df_aux["month"].str.replace("sNovember","November")
df_aux["month"] = df_aux["month"].str.replace("sSep","Sep")
df_aux["month"] = df_aux["month"].str.replace("yAug","Aug")

In [21]:
[df_aux["month"].iloc[x] for x in list1]

['Feb', 'Aug', 'November', 'Oct', 'November', 'Sep', 'Aug']

### We now have a clean data frame but as the last step converting everything to numeric

In [22]:
#Replace all the months text with numbers:

df_aux["month"].replace({'Jan': '1', 'January': '1',
                         'Feb': '2', 'February': '2',
                         'Mar': '3', 'March': '3',
                         'Apr': '4', 'April': '4',
                         'May': '5',
                         'Jun': '6', 'June': '6',
                         'Jul': '7', 'July': '7',
                         'Aug': '8', 'August': '8',
                         'Sep': '9', 'September': '9',
                         'Oct': '10', 'October': '10',
                         'Nov': '11', 'November': '11',
                         'Dec': '12', 'December': '12',}, inplace = True)

#Convert everything to integers (the column "year" was converted previously):

df_aux = df_aux.astype({"month":'int'})
df_aux = df_aux.astype({"day":'int'})

#Check that all the data types are accurate:
df_aux.dtypes

month    int64
day      int64
year     int64
dtype: object

### Final Dataset

The final data frame manual review

In [23]:
cols = ["year", "month", "day"]
df.loc[df_aux.index, cols] = df_aux[cols]

df

/var/folders/_c/52mb0p3d04b0btfj7p3ymtn40000gn/T/ipykernel_7419/2841603985.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df_aux.index, cols] = df_aux[cols]


,text,year,month,day
0,03/25/93 Total time of visit (in minutes):\n,1993,3,25
1,6/18/85 Primary Care Doctor:\n,1985,6,18
2,sshe plans to move as of 7/8/71 In-Home Services: None\n,1971,7,8
3,7 on 9/27/75 Audit C Score Current:\n,1975,9,27
4,2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n,1996,2,6
5,.Per 7/06/79 Movement D/O note:\n,1979,7,6
6,"4, 5/18/78 Patient's thoughts about current substance abuse:\n",1978,5,18
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n,1989,10,24
8,3/7/86 SOS-10 Total Score:\n,1986,3,7
9,(4/10/71)Score-1Audit C Score Current:\n,1971,4,10


### After manual review, it seems everything is fine. Exporting the data frame to excel

In [24]:
df.to_excel("processed_dates.xlsx", index=False)

## Part 2 (plotly)

Explore using plotly / plotly express the following questions:

* Is there any relation between neighborhood and price?

* Is there any relation between neighborhood and year built?

* How overall quality, lot area, year built and price interact with each other?

* How quality, lot area, year built and price interact with each other and evolve in time?

In [25]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
py.init_notebook_mode(connected=True) # display plotly graphs in notebook

In [26]:
df_px = pd.read_csv("dataset_housing.csv")
df_px.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Is there any relation between neighborhood and price?

There is a clear relationship between these 2 variables with NoRidge being the most expensive neighbourhood on an average and MeadowV the cheapest

In [27]:
df_neighbourhood_SalePrice=pd.DataFrame(df_px.groupby('Neighborhood')['SalePrice'].mean()).reset_index()

In [28]:
fig = px.bar(df_neighbourhood_SalePrice.sort_values(by='SalePrice', ascending=False), x="Neighborhood", y="SalePrice")
fig.show()

### Visualising using scatter

In [29]:
fig = px.scatter(df_px, x="Neighborhood", y="SalePrice")
fig.update_layout(title_text = 'Sale Price by Neighborhood')
fig.show()

### Is there any relation between neighborhood and year built?

- There seems to be a relation here as well. Neighbourhoods like Oldtown mostly have old houses and few outliers wherein houses are built after 2000 and median 1920. Noridge which is the most expensive neighbourhood has houses built after 1990

In [41]:
df_px.shape

(1460, 83)

In [31]:
fig = px.box(df_px, x="Neighborhood", y="YearBuilt")
fig.update_layout(title_text = 'Year Built by Neighborhood')
fig.show()

### How overall quality, lot area, year built and price interact with each other?


- **YearBuilt and OverallQuall**: Newer Houses seem to have a better OverallQuall
- **SalePrice and OverallQuall**: This one is quite obvious with a clear positive correlation.
- **LotArea and OverallQuall**: difficult to infer a relationship.
- **SalePrice and YearBuilt**: Newer houses seem to be more expensive which also ties in with the overall quality.
- **LotArea and YearBuilt**: difficult to infer a relationship.
- **LotArea and SalePrice**: Positive relationship here and with a bigger house the sale price tends to go higher. 

In [32]:
fig = px.scatter_matrix(df_px[["OverallQual","YearBuilt","SalePrice","LotArea"]], opacity = 0.1)
fig.update_traces(diagonal_visible=False)
fig.update_layout(title_text = 'Interaction of OverallQual, YearBuilt, SalePrice and LotArea')
fig.show()

/Users/mukhtadirsyed/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [33]:
# We transform LotArea and SalePrice to log to have a clearer picture of the relationships.
df_px["LotArea_log"] = np.log(df_px["LotArea"])
df_px["SalePrice_log"] = np.log(df_px["SalePrice"])

fig = px.scatter_matrix(df_px[["OverallQual","YearBuilt","SalePrice_log","LotArea_log"]])
fig.update_traces(diagonal_visible=False)
fig.update_layout(title_text = 'Interaction of OverallQual, YearBuilt, SalePrice and LotArea')
fig.show()

/Users/mukhtadirsyed/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



### How quality, lot area, year built and price interact with each other and evolve in time?

In this graph, we can see how all the variables interact with each other through time. Some remarks:

- Through the years newer houses have a higher sale price (year built and sale price).
- There is a clear positive relationship between OverallQuall and SalePrice through the years.
- We can see that, usually, the houses with the highest sale prices for each year, have a big LotArea number (arround 13k square feets). This too validates our assumption that newer and bigger houses command a higher price

In [42]:
#Sort values by YrSold for the animated frame
df_px = df_px.sort_values(by = "YrSold")

fig = px.scatter(
    df_px, x="SalePrice", y="OverallQual", animation_frame="YrSold",
    animation_group="Neighborhood", size="LotArea", color="YearBuilt",
    hover_name="Neighborhood", size_max=75, range_x=[0, 400000])
fig.update_layout(title_text = "Interaction of OverallQual, YearBuilt, SalePrice and LotArea through the years (bubblesize = LotArea)")
py.iplot(fig)